# Functions

## Import libraries

In [1]:
import itertools
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import reduce
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from cvxopt import matrix, solvers

## Data preparation

In [2]:
def prepare_data_q23():
    '''
    This function downloads and prepare the data for the question 2 and 3.

    Since the data was originally uploaded on the Classroom page of the course
    and Colab is not able to access it, the .csv file has been re-uploaded
    unaltered to the Google Drive of my student account and made public for
    Colab to access.

    For the train-test split it has been choosen the classic 80/20 split and the
    random state has been fixed to ensure reproducibility of the results even
    if the funciton is called at different times throughout the notebook.
    '''

    # download gender classification data
    !gdown 19v7WEWtHEpgTDr7PZwNGQNWa5u6tB0Cb -q

    # load data into Pandas
    df = pd.read_csv('/content/GENDER_CLASSIFICATION.csv')

    # convert data to NumPy data structures
    x = df.iloc[:, 0:32].to_numpy()
    y = df.iloc[:, 32].to_numpy().reshape(-1, 1)

    # converts classes of 1 and 0 to classes of 1 and -1
    # which is required for SVM classification
    y = 2 * y - 1

    # split data into training and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

    # normalize data
    scaler = StandardScaler()

    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    return x_train, x_test, y_train, y_test

def prepare_data_q4():
    '''
    This function downloads and prepare the data for the question 4.

    Since the data was originally uploaded on the Classroom page of the course
    and Colab is not able to access it, the .csv file has been re-uploaded
    unaltered to the Google Drive of my student account and made public for
    Colab to access.

    For the train-test split it has been choosen the classic 80/20 split and the
    random state has been fixed to ensure reproducibility of the results.

    The 3 classes selected out of 5 for the task are the first 3.
    '''

    # download ethnicity classification data
    !gdown 1M_h_L7TM7-biHezB3jbHC5zAnK5ax9SP -q

    # load data into Pandas
    df = pd.read_csv('/content/ETHNICITY_CLASSIFICATION.csv')

    # select only three classes are requested by the question
    df = df[df['gt'].isin([0, 1, 2])]

    # convert data to NumPy data structures
    x = df.iloc[:, 0:32].to_numpy()
    y = df.iloc[:, 32].to_numpy().reshape(-1, 1)

    # split data into training and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

    # normalize data
    scaler = StandardScaler()

    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    return x_train, x_test, y_train, y_test

## Auxiliary functions

In [3]:
def gaussian_kernel(x, y, gamma):
    '''
    This function implements the Gaussian kernel.

    For the implementation we just used the formula provided:
    K(x, y) = exp(- gamma * ||x - y||^2)

    We implement it in a vectorized way since we will need it to take both X
    and Y as 2D arrays for the construction of the K matrix and one 1D and the
    other 2D for the inference.
    '''

    # if x is 1D reshape to 2D
    if x.ndim == 1:
        x = x.reshape(-1, 1)

    # if x is 1D reshape to 2D
    if y.ndim == 1:
        y = y.reshape(-1, 1)

    # vectorized euclidean distances matrix
    dists = np.sum((x[:, np.newaxis, :] - y[np.newaxis, :, :])**2, axis = -1)

    # compute kernel
    K = np.exp(- gamma * dists)

    # if both x and y were single samples, return scalar
    if K.size == 1:
        return K.item()

    # flatten if result is 1D array
    elif K.ndim == 2 and (K.shape[0] == 1 or K.shape[1] == 1):
        return K.flatten()

    # else return result matrix
    else:
        return K

## Question 2

### Support Vector Machine

In [4]:
class SVM():
    '''
    This class is an implementation of a Support Vector Machine.

    The input to the class constructor are the kernel function, the parameter
    of the kernel function and the value C, the upper bound of the constraints
    of the dual problem.

    The optimization procedure finds the support vectors and the bias while the
    hyperparameters are to be found externally through cross-validation.

    The training of the SVM is carried out as an optimization problem, we
    express the objective function and the constraints of the dual problem as
    a convex quadratic programming problem and solve it numerically using the
    library CVXOPT.
    '''

    def __init__(self, kernel_function, kernel_parameter, C, x, y):
        # store the parameters of the model inside the object
        self.kernel_function = globals().get(kernel_function)
        self.kernel_parameter = kernel_parameter
        self.C = C

        # store the training data inside the object
        self.x = x
        self.L = x.shape[0]
        self.y = y

        # initial guess for lambda vector
        self.lam = np.zeros((self.L))

        # initialize K and Q matrices
        self.K = self.build_K_matrix()
        self.Q = self.build_Q_matrix()

    def build_K_matrix(self):
        '''
        This function creates a matrix of size L x L such that:

        K[i, j] = kernel_function(x_i, x_j)

        This matrix is used to build the Q matrix but also to calculate the bias
        at the end of the training.
        '''

        return self.kernel_function(self.x, self.x, self.kernel_parameter)

    def build_Q_matrix(self):
        """
        This function creates a matrix of size L x L such that:

        Q[i, j] = y_i * y_i * kernel_function(x_i, x_j)

        This is the matrix Q in the objective function of the SVM problem.
        """

        return self.K * np.outer(self.y, self.y)

    def objective(self):
        '''
        This function calculates the objective function of the dual problem
        using the current self.lam values. Objective function is:

        (1/2) \lambda^T Q \lambda - e^T \lambda

        This function is not needed for the training or the inference but the
        homework require us to report its value before and after the training
        '''

        return (1/2) * np.dot(np.dot(self.lam.T, self.Q), self.lam) - np.sum(self.lam)

    def wks(self, x):
        '''
        This function implements what we call the "weighted kernel sum", we gave
        it this name because it is the sum of the kernel function between the
        input vector x and every support vector multiplied by the sign (class)
        of the support vector and weighted by the corresponding lambda.

        It implements the formula:

        \sum_{j \in SV} \lambda_j y_j K(x_j, x)

        Where SV is the set of support vectors and x is the input.

        The function has been vectorized to accept x as a 2D array of shape
        (n_features, n_samples) to avoid the use of loops or list comprehensions.
        '''

        return np.sum(self.sv_lam.reshape(-1, 1) * self.sv_y.reshape(-1, 1) * self.kernel_function(self.sv_x, x, self.kernel_parameter), axis = 0)

    def optimize(self):
        '''
        This function sets up the matrix components of the optimization problem
        and solves it through the CVXOPT library.

        Initialization of the components follow the reasonings made in the report
        to translate the soft-margin SVM into the form required by CVXOPT.
        '''

        # initialize problem components

        P = matrix(self.Q)
        q = matrix(-np.ones((self.L, 1)).astype(float))

        G = matrix(np.vstack((-np.eye(self.L), np.eye(self.L))).astype(float))
        h = matrix(np.vstack((np.zeros((self.L, 1)), self.C * np.ones((self.L, 1)))).astype(float))

        A = matrix(self.y.T.astype(float))
        b = matrix(float(0))

        # initial guess
        x0 = matrix(self.lam)

        # start measuring time
        time_init = time.process_time()

        # solve the convex optimization problem
        sol = solvers.qp(P, q, G, h, A, b, initvals={'x': x0})

        # stop measuring time and compute CPU time of optimization
        time_end = time.process_time()
        time_delta = time_end - time_init

        return sol, time_delta

    def build_svm_params(self):
        '''
        From the dual solution recovers the solution of the primal. The support
        vectors are all the samples which lambda is greater than 0.
        '''

        # as per definition support vectors are all those non-zero lambdas

        # we are using the mask only for computational efficiency, if the lambda
        # is zero the component has no weight on the result but efficiency is lost
        sv_mask = self.lam.flatten() > 0

        # for inference we need only the x, y and lambda of support vectors
        self.sv_x = self.x[sv_mask]
        self.sv_y = self.y[sv_mask].flatten()
        self.sv_lam = self.lam[sv_mask]

        # calculate bias
        # here we are using the formula of the bias provided by the lectures
        # b^* = 1 / |SV| \sum_{h \in SV} (y^h - \sum_{j = 1}^{l} \lambda_j y_j K(x_j, x_h))
        self.sv_bias = np.mean(self.sv_y - np.dot(self.K[np.where(sv_mask)[0]], self.lam.reshape(-1, 1) * self.y))

    def train(self, verbose = False):
        '''
        From the dual solution recovers the solution of the primal computing

        The support vectors are all the samples which lambda is greater than 0,
        considering a value for tolerance.
        '''

        # solve optimization problem and get solution
        sol, time_delta = self.optimize()

        # update multipliers
        self.lam = np.array(sol['x']).flatten()

        # create parameters for inference
        self.build_svm_params()

        if verbose:
            return sol['iterations'], time_delta
        else:
            return

    def inference(self, x):
        '''
        This function takes as input an unseen vector x and makes a prediction
        of its class using the model built during the training.

        The function has been vectorized to accept x as a 2D array of shape
        (n_features, n_samples) to avoid the use of loops or list comprehensions.
        '''

        return np.sign(self.wks(x) + self.sv_bias).reshape(-1, 1)

    def accuracy(self, x, y):
        '''
        This function calculates the accuracy of the model.

        accuracy = correctly predicted / total
        '''

        return np.sum(self.inference(x) == y) / y.shape[0]

    def confusion_matrix(self, x, y):
        '''
        This function calculates the confusion matrix from the provided data.
        '''

        # get subsets of samples
        x_class1 = x[y.flatten() == 1]
        x_class2 = x[y.flatten() == -1]

        # get predictions for the subsets
        y_class1 = self.inference(x_class1)
        y_class2 = self.inference(x_class2)

        # create empty confusion matrix
        confusion_matrix = np.zeros((2, 2))

        # fill confusion matrix
        confusion_matrix[0, 0] = np.sum(y_class1 == 1)
        confusion_matrix[0, 1] = np.sum(y_class1 == -1)

        confusion_matrix[1, 0] = np.sum(y_class2 == 1)
        confusion_matrix[1, 1] = np.sum(y_class2 == -1)

        return confusion_matrix

### Cross-validation

In [5]:
def SVM_cross_validation(kernel_functions, kernel_parameters, C_values, k, show_progress = True):
    '''
    This function implements the cross-validation for the SVM hyperparameters.

    It creates the list of all possible combination of hyperparameters and for
    each combination trains k models using each time a different fold as
    validation set.

    Performances are measured and averaged across all folds and the best
    combination of hyperparameters is returned, together with its performances.
    '''

    # load data question 2-3
    x, _, y, _ = prepare_data_q23()

    # generate all possible combinations of kernel structures, kernel parameters and C values
    combinations = list(itertools.product(kernel_functions, kernel_parameters, C_values))

    # split train data into folds
    k_folds = KFold(n_splits = k)

    # initialize variables to find best combination
    best_val_accuracy = 0
    best_train_accuracy = 0
    best_combination = None

    for combination in tqdm(combinations) if show_progress else combinations:
        fold_val_accuracy = []
        fold_train_accuracy = []

        for train_index, val_index in k_folds.split(x):
            # split data into train and validation sets
            x_train = x[train_index]
            y_train = y[train_index]

            x_val = x[val_index]
            y_val = y[val_index]

            # initialize and train model
            np.random.seed(42)
            model = SVM(combination[0], combination[1], combination[2], x_train, y_train)
            model.train()

            # evaluate model
            fold_val_accuracy.append(model.accuracy(x_val, y_val))
            fold_train_accuracy.append(model.accuracy(x_train, y_train))

        # compute combination accuracy as the mean of the MAPE for all folds
        mean_val_accuracy = np.mean(fold_val_accuracy)
        mean_train_accuracy = np.mean(fold_train_accuracy)

        # update best combination if current combination is better
        if mean_val_accuracy > best_val_accuracy:
            best_val_accuracy = mean_val_accuracy
            best_train_accuracy = mean_train_accuracy
            best_combination = combination

    return best_combination, best_val_accuracy, best_train_accuracy

## Question 3

In [6]:
class MVP():
    '''
    This class is an implementation of a Support Vector Machine trained using
    the Sequential Minimal Optimization (SMO) decomposition method where the
    Working Set Selection Rule (WSSR) is the Most Violating Pair (MVP).

    Until the optimality condition is satisfied we calculate the Most Violating
    Pair, solve the subproblem analytically and use the result of the
    subproblem to update the multipliers, the gradient and the bias.
    '''

    def __init__(self, kernel_function, kernel_parameter, C, x, y, opt_tol = 1e-1):
        # store model paramenters inside the object
        self.kernel_function = globals().get(kernel_function)
        self.kernel_parameter = kernel_parameter
        self.C = C
        self.opt_tol = opt_tol

        # store training data
        self.x = x
        self.L = x.shape[0]
        self.y = y

        # initialize interaction matrix
        # it is useful to calculate it once and store it as it will called
        # multiple times throughout the training algorihtm
        self.K = self.build_K_matrix()
        self.Q = self.build_Q_matrix()

        # initial guess for a vector
        self.a = np.zeros((self.L))

        # initialize gradients as vector of -1
        self.gradient = - np.ones((self.L, 1))

        # initialize bias
        self.sv_bias = 0

    def build_K_matrix(self):
        '''
        This function creates a matrix of size L x L such that:

        K[i, j] = kernel_function(x_i, x_j)

        This matrix is used to build the Q matrix but also to calculate the bias
        at the end of the training. Here in the MVP algorithm it is also used
        for solving the subproblems.
        '''

        return self.kernel_function(self.x, self.x, self.kernel_parameter)

    def build_Q_matrix(self):
        """
        This function creates a matrix of size L x L such that:

        Q[i, j] = y_i * y_i * kernel_function(x_i, x_j)

        This is the matrix Q in the objective function of the SVM problem.
        """

        return self.K * np.outer(self.y, self.y)

    def objective(self):
        '''
        This function calculates the objective function of the dual problem
        using the current self.a values.
        '''

        return (1/2) * np.dot(np.dot(self.a.T, self.Q), self.a) - np.sum(self.a)

    def wks(self, x):
        '''
        This function implements what we call the "weighted kernel sum", we gave
        it this name because it is the sum of the kernel function between the
        input vector x and every support vector multiplied by the sign (class)
        of the support vector and weighted by the corresponding lambda.

        It implements the formula:

        \sum_{j \in SV} \lambda_j y_j K(x_j, x)

        Where SV is the set of support vectors and x is the input.

        The function has been vectorized to accept x as a 2D array of shape
        (n_features, n_samples) to avoid the use of loops or list comprehensions.
        '''

        return np.sum(self.sv_lam.reshape(-1, 1) * self.sv_y.reshape(-1, 1) * self.kernel_function(self.sv_x, x, self.kernel_parameter), axis = 0)

    def descent_direction(self, i):
        '''
        This function computes - \nabla_i f(a) / y_i, which is needed to find
        the Most Violating Pair.
        '''

        return - self.gradient[i] / self.y[i]

    def find_R_S(self, tol = 1e-8):
        '''
        This function calculates the two sets R and S needed for finding the
        Most Violating Pair and for the optimality check.

        Here we are simply applying the definitions provided by the lectures
        to create the sets.

        Even though this is an analytical solution computer precision may still
        provide small errors, therefore we need to add a tolerance to consider
        a value as 0 or C.
        '''

        # {i = 1,..,l : a_i = 0 \and y^i = -1}
        L_minus = np.where((self.a <= 0 + tol) & (self.y.flatten() == -1))[0]

        # {i = 1,..,l : a_i = 0 \and y^i = 1}
        L_plus = np.where((self.a <= 0 + tol) & (self.y.flatten() == 1))[0]

        # {i = 1,..,l : a_i = C \and y^i = -1}
        U_minus = np.where((self.a >= self.C - tol) & (self.y.flatten() == -1))[0]

        # {i = 1,..,l : a_i = C \and y^i = 1}
        U_plus = np.where((self.a >= self.C - tol) & (self.y.flatten() == 1))[0]

        # {i : 0 < a_i < C}
        middle = np.where((self.a > 0 + tol) & (self.a < self.C - tol))[0]

        # L_plus U U_minus U middle
        R = reduce(np.union1d, (L_plus, U_minus, middle))

        # L_minus U U_plus U middle
        S = reduce(np.union1d, (L_minus, U_plus, middle))

        return R, S

    def check_optimality(self, verbose = False):
        '''
        This function implements the check of the condition to stop the
        optimization procedure. We say that a^k is optimal if:

        max_{i \in R(a^k)}{- \nabla_i f(a^k) / y_i } <= min_{j \in S(a^k)}{- \nabla_j f(a^k) / y_j}

        It is proved that SMO-MVP gets to an optimal solution in a finite number
        of iterations however to avoid long computation times we add a tolerance
        to the condition.
        '''

        R, S = self.find_R_S()

        R_set = [self.descent_direction(i) for i in R]
        S_set = [self.descent_direction(j) for j in S]

        if len(R_set) == 0:
            m = - float('inf')
        else:
            m = np.max(R_set)

        if len(S_set) == 0:
            M = float('inf')
        else:
            M = np.min(S_set)

        if verbose:
            return m, M
        else:
            if m <= (M + self.opt_tol):
                return True
            else:
                return False

    def difference(self):
        """
        This function calculates the difference between m and M, which is needed
        for the report. When the difference is negative the optimality condition
        is satisfied.
        """

        m, M = self.check_optimality(verbose = True)

        return m - M - self.opt_tol

    def find_mvp(self):
        '''
        This function finds the Most Violating Pair. It is the set of indices
        {I, J} such that:

        I(a^k) = argmax_{i \in R(a^k)}{- \nabla_i f(a^k) \ y_i}
        J(a^k) = argmin_{j \in S(a^k)}{- \nabla_j f(a^k) \ y_j}
        '''

        R, S = self.find_R_S()

        R_set = [(i, self.descent_direction(i)) for i in R]
        S_set = [(j, self.descent_direction(j)) for j in S]

        I = max(R_set, key = lambda z: z[1])[0]
        J = min(S_set, key = lambda z: z[1])[0]

        return I, J

    def compute_E(self, i):
        """
        This function computes the difference between the current value of the
        prediction for the vector x_i before the sign function and the true
        value of the prediction.
        """

        return (np.dot(self.K[i], self.a.reshape(-1, 1) * self.y) + self.sv_bias - self.y[i])[0]

    def solve_subproblem(self, i, j):
        """
        Solves the quadratic programming subproblem of size 2. The algorithm
        uses the transposition into code of what is described mathematically in
        the 1998 paper by John C. Platt "Sequential Minimal Optimization: A Fast
        Algorithm for Training Support Vector Machines", the paper that firstly
        proposed the SMO algorithm. The paper is available here:

        https://www.microsoft.com/en-us/research/uploads/prod/1998/04/sequential-minimal-optimization.pdf
        """

        a_new = [None, None]

        # get upper and lower bounds that satisfy the constraints
        if self.y[i] != self.y[j]:
            L = np.max([0, self.a[j] - self.a[i]])
            H = np.min([self.C, self.C - self.a[i] + self.a[j]])

        else:
            L = np.max([0, self.a[i] + self.a[j] - self.C])
            H = np.min([self.C, self.a[i] + self.a[j]])

        # get E values for both i and j
        Ei = self.compute_E(i)
        Ej = self.compute_E(j)

        # get new unconstrained optimizing value for aj
        a_new[1] = self.a[j] + self.y[j] * (Ej - Ei) / (- self.K[i, i] - self.K[j, j] + 2 * self.K[i, j])

        # clip aj to get optimizing value that enforces the constraints
        a_new[1] = np.clip(a_new[1], L, H)[0]

        # compute ai from aj, this is forced by equality constraint
        a_new[0] = (self.a[i] + self.y[i] * self.y[j] * (self.a[j] - a_new[1]))[0]

        # update the bias
        # when both conditions are valid bi and bj are equal, when none is valid
        # all thresholds are consistent and as done by the paper we select it
        # halfway between the two

        if a_new[0] > 0 and a_new[0] < self.C:
            self.sv_bias = self.sv_bias - Ei - self.y[i] * (a_new[0] - self.a[i]) * self.K[i, i] - self.y[j] * (a_new[1] - self.a[j]) * self.K[i, j]

        elif a_new[1] > 0 and a_new[1] < self.C:
            self.sv_bias = self.sv_bias - Ej - self.y[i] * (a_new[0] - self.a[i]) * self.K[i, j] - self.y[j] * (a_new[1] - self.a[j]) * self.K[j, j]

        else:
            bi = self.sv_bias - Ei - self.y[i] * (a_new[0] - self.a[i]) * self.K[i, i] - self.y[j] * (a_new[1] - self.a[j]) * self.K[i, j]
            bj = self.sv_bias - Ej - self.y[i] * (a_new[0] - self.a[i]) * self.K[i, j] - self.y[j] * (a_new[1] - self.a[j]) * self.K[j, j]
            self.sv_bias = (bi + bj) / 2

        return a_new

    def optimization_step(self):
        '''
        Find the Most Violating Pair, compute a_i and a_j solving the quadratic
        subproblem, update a, compute the new gradient vector.
        '''

        # find the most violating pair for the current a
        i, j = self.find_mvp()

        ai_old = self.a[i]
        aj_old = self.a[j]

        # start measuring time
        time_init = time.process_time()

        # solve the convex optimization problem finding the new values for
        # a_i and a_j

        self.a[i], self.a[j] = self.solve_subproblem(i, j)

        # stop measuring time and compute CPU time of optimization
        time_end = time.process_time()
        time_delta = time_end - time_init

        # compute and memorize the new gradient that will be needed for new
        # iterations and optimality checks

        self.gradient = self.gradient + self.Q[:,i].reshape(-1, 1) * (self.a[i] - ai_old) + self.Q[:,j].reshape(-1, 1) * (self.a[j] - aj_old)

        return time_delta

    def build_svm_params(self):
        '''
        From the dual solution recovers the solution of the primal. The support
        vectors are all the samples which lambda is greater than 0.
        '''

        # as per definition support vectors are all those non-zero lambdas

        # we are using the mask only for computational efficiency, if the lambda
        # is zero the component has no weight on the result but efficiency is lost
        sv_mask = self.a.flatten() > 0

        # for inference we need only the x, y and lambda of support vectors
        self.sv_x = self.x[sv_mask]
        self.sv_y = self.y[sv_mask].flatten()
        self.sv_lam = self.a[sv_mask]

        # calculate bias
        # here we are using the formula of the bias provided by the lectures
        # b^* = 1 / |SV| \sum_{h \in SV} (y^h - \sum_{j = 1}^{l} \lambda_j y_j K(x_j, x_h))
        self.sv_bias = np.mean(self.sv_y - np.dot(self.K[np.where(sv_mask)[0]], self.a.reshape(-1, 1) * self.y))

    def train(self, verbose = False):
        '''
        From the dual solution recovers the solution of the primal finding
        the parameters lambda and b.

        The support vectors are all the lambda that are greater than 0,
        considering a value for tolerance.
        '''

        # initialize iteration counters
        iterations = 0
        time_delta = 0

        while not self.check_optimality():
            # execute optimization step
            current_time_delta = self.optimization_step()

            # update counters
            iterations += 1
            time_delta += current_time_delta

        self.build_svm_params()

        if verbose:
            return iterations, time_delta
        else:
            return

    def inference(self, x):
        '''
        This function takes as input an unseen vector x and makes a prediction
        of its class using the model built during the training.

        The function has been vectorized to accept x as a 2D array of shape
        (n_features, n_samples) to avoid the use of loops or list comprehensions.
        '''

        return np.sign(self.wks(x) + self.sv_bias).reshape(-1, 1)

    def accuracy(self, x, y):
        '''
        This function calculates the accuracy of the model.

        accuracy = correctly predicted / total
        '''

        return np.sum(self.inference(x) == y) / y.shape[0]

## Question 4

In [7]:
class SVM_multiclass():
    '''
    This class is an implementation of a multiclass classification using
    Support Vector Machines.

    To implement the multiclass classification it has been used a One-vs-One
    approach and the choosen classification is selected by majority voting.

    To increase computation speed the SVM is trained using SMO-MVP
    '''

    def __init__(self, kernel_function, kernel_parameter, C, x, y):
        # store model paramenters inside the object
        self.kernel_function = kernel_function
        self.kernel_parameter = kernel_parameter
        self.C = C

        # store training data
        self.x = x
        self.L = self.x.shape[0]
        self.y = y

        # set of classes the classificator has to deal with
        self.classes = np.unique(y)

    def train(self, verbose = False):
        '''
        This function implements the training for the multiclass classification.

        For each combination of classes it is created, trained and stored a
        Support Vector Machine.
        '''

        # initialize SVM list
        self.SVMs = []

        # iterations
        iterations = 0
        time_delta = 0

        # cycles through each combination of classes
        for class1, class2 in itertools.combinations(self.classes, 2):
            # create mask for the data of the two selected ckasses
            mask = np.logical_or(self.y == class1, self.y == class2).flatten()

            # apply mask on train data
            x_train = self.x[mask, :]
            y_train = self.y[mask]

            # convert classification value to -1 and +1 as SVM use this
            # kind of classification
            y_train[y_train == class1] = -1
            y_train[y_train == class2] = 1

            # create and train the SVM
            model = MVP(self.kernel_function, self.kernel_parameter, self.C, x_train, y_train)
            current_iterations, current_time_delta = model.train(verbose = True)

            # update iterations
            iterations += current_iterations
            time_delta += current_time_delta

            # insert trained SVM into the list together with identifiers of the
            # two classes the SVM classifies
            self.SVMs.append(((class1, class2), model))

        if verbose:
            return iterations, time_delta
        else:
            return

    def inference(self, x):
        '''
        This function implements the inference for the multiclass classification.

        The function has been vectorized to accept x as a 2D array of shape
        (n_features, n_samples) to avoid the use of loops or list comprehensions.

        It creates a matrix where votes for each sample for each class are
        stored, it populates the matrix using the inference on every SVM inside
        the model and then it returns the class with the most votes.
        '''

        # initialize votes matrix
        votes = np.zeros((x.shape[0], len(self.classes)))

        # iterate through all the trained SVMs
        for (class1, class2), SVM in self.SVMs:

            # get predictions from the SVM
            predictions = SVM.inference(x)

            # convert prediction to masks
            predictions_class1 = (predictions == -1)
            predictions_class2 = (predictions == 1)

            # update votes matrix
            votes[:, np.where(self.classes == class1)[0]] += predictions_class1
            votes[:, np.where(self.classes == class2)[0]] += predictions_class2

        # return predictions
        return self.classes[np.argmax(votes, axis=1)].reshape(-1, 1)

    def accuracy(self, x, y):
        return round(np.sum(self.inference(x) == y) / y.shape[0], 3)

## Required prints, values and plots to compile the report

In [8]:
def SVM_print(kernel_function, kernel_parameter, C):

    print(f"Setting values of the hyperparameters:")
    print(f"\tC = {C}")
    print(f"\tActivation function = {kernel_function}")
    print(f"\tgamma = {kernel_parameter}")

    # load data question 2-3
    x_train, x_test, y_train, y_test = prepare_data_q23()

    # initialize and train model
    np.random.seed(42)
    model = SVM(kernel_function, kernel_parameter, C, x_train, y_train)
    iterations, time_delta = model.train(verbose = True)
    objective = model.objective()

    print(f"Classification rate on the training set (% instances correctly classified): {model.accuracy(x_train, y_train)}")
    print(f"Classification rate on the test set (% instances correctly classified): {model.accuracy(x_test, y_test)}")

    # The confusion matrix
    print(f"The confusion matrix:")
    print(f"{model.confusion_matrix(x_test, y_test)}")

    print(f"Time necessary for the optimization procedure: {time_delta} seconds")
    print(f"Number of optimization iterations: {iterations}")

    # Final difference between m(λ) and M(λ) (only for question 3)
    np.random.seed(42)
    model = MVP(kernel_function, kernel_parameter, C, x_train, y_train)
    model.train()
    difference = model.difference()
    print(f"Final difference between m(λ) and M(λ) (question 3): {difference}")

    print(f"Final value of the objective function of the dual SVM problem: {objective}")

In [9]:
def SVM_report(kernel_function, kernel_parameter, C):

    # load data question 2-3
    x_train, x_test, y_train, y_test = prepare_data_q23()

    # initialize SVM
    np.random.seed(42)
    model = SVM(kernel_function, kernel_parameter, C, x_train, y_train)

    # get initial objective
    initial_objective = model.objective()

    # train model and get values to report
    iterations, time_delta = model.train(verbose = True)

    final_objective = model.objective()

    # the final setting for the hyperparameter C (upper bound of the constraints
    # of the dual problem) and of the hyperparameter of the kernel chosen; how
    # you have chosen them and if you could identify values that highlight
    # over/underfitting;
    print(f"Setting values of the hyperparameters:")
    print(f"\tKernel function = {kernel_function}")
    print(f"\tC = {C}")
    print(f"\tgamma = {kernel_parameter}")
    # identify values that highlight overfitting/underfitting?

    # which optimization routine you use for solving the quadratic minimization
    # problem and the setting of its parameters, if any (write DEFAULT if you
    # have not changed them);
    print(f"Optimization routine used: cvxopt.solvers.qp()")
    print(f"Optimization routine parameters:")

    # write the value of the accuracy on the training, validation and test set
    np.random.seed(42)
    _, best_val_accuracy, _ = SVM_cross_validation([kernel_function], [kernel_parameter], [C], k = 5, show_progress = False)
    print(f"Accuracy of SVM on train set: {model.accuracy(x_train, y_train)}")
    print(f"Accuracy of SVM on validation set: {best_val_accuracy}")
    print(f"Accuracy of SVM on test set: {model.accuracy(x_test, y_test)}")

    # report the initial and final value of the objective function of the dual
    # problem and the number of iterations
    print(f"Initial value of the objective function of the dual SVM problem: {initial_objective}")
    print(f"Final value of the objective function of the dual SVM problem: {final_objective}")
    print(f"Number of optimization iterations: {iterations}")
    print(f"Time necessary for the optimization procedure: {time_delta} seconds")
    print("\n")

    # initialize MVP SVM
    np.random.seed(42)
    model = MVP(kernel_function, kernel_parameter, C, x_train, y_train)

    # get initial objective
    initial_objective = model.objective()

    # train model
    iterations, time_delta = model.train(verbose = True)

    # get final objective
    final_objective = model.objective()

    # write the value of the accuracy on the training and test set
    print(f"Accuracy of MVP SVM on train set: {model.accuracy(x_train, y_train)}")
    print(f"Accuracy of MVP SVM on test set: {model.accuracy(x_test, y_test)}")

    # report the initial and final value of the objective function of the dual
    # problem and the number of iterations
    print(f"Initial value of the objective function of the MVP problem: {initial_objective}")
    print(f"Final value of the objective function of the MVP problem: {final_objective}")
    print(f"Number of iterations: {iterations}")
    print(f"Time necessary for the optimization procedure: {time_delta} seconds")

    # TABLE
    print(f"Kernel function: {kernel_function}")
    print(f"C: {C}")
    print(f"Kernel parameter: {kernel_parameter}")
    print("\n")

    # load data question 4
    x_train, x_test, y_train, y_test = prepare_data_q4()

    # initialize multiclass SVM
    np.random.seed(42)
    model = SVM_multiclass(kernel_function, kernel_parameter, C, x_train, y_train)

    # train model
    iterations, time_delta = model.train(verbose = True)

    print(f"Accuracy of multiclass SVM on train set: {model.accuracy(x_train, y_train)}")
    print(f"Accuracy of multiclass SVM on test set: {model.accuracy(x_test, y_test)}")

    print(f"Number: {iterations}")
    print(f"Time necessary for the optimization procedure: {time_delta} seconds")

In [10]:
def plot_varying_gamma(kernel_function, gammas, C):
    """
    Trains multiple models with fixed C and varying gamma, then plot the training
    and test accuracy to show overfitting and underfitting.
    """

    # load data question 2-3
    x_train, x_test, y_train, y_test = prepare_data_q23()

    train_accs = []
    test_accs = []

    for gamma in tqdm(gammas):
        # initialize and train model
        model = SVM(kernel_function, gamma, C, x_train, y_train)
        model.train()

        # compute accuracies
        train_accs.append(model.accuracy(x_train, y_train))
        test_accs.append(model.accuracy(x_test, y_test))

    # plot
    plt.figure(figsize=(10, 6))
    plt.plot(gammas, train_accs, label='Train Accuracy', marker='o')
    plt.plot(gammas, test_accs, label='Test Accuracy', marker='o')
    plt.xscale('log')
    plt.xlabel('Gamma')
    plt.ylabel('Accuracy')
    plt.title('Train and Test Accuracy vs. Gamma (Fixed C)')
    plt.legend()
    plt.grid(True)
    plt.show()

def ploy_varying_C(kernel_function, gamma, C_values):
    """
    Trains multiple models with fixed gamma and varying C, then plots the training
    and test accuracy to show overfitting and underfitting.
    """

    # load data question 2-3
    x_train, x_test, y_train, y_test = prepare_data_q23()

    train_accs = []
    test_accs = []

    for C in tqdm(C_values):
        # initialize and train model
        model = SVM(kernel_function, gamma, C, x_train, y_train)
        model.train()

        # compute accuracies
        train_accs.append(model.accuracy(x_train, y_train))
        test_accs.append(model.accuracy(x_test, y_test))

    # plot
    plt.figure(figsize=(10, 6))
    plt.plot(C_values, train_accs, label='Train Accuracy', marker='o')
    plt.plot(C_values, test_accs, label='Test Accuracy', marker='o')
    plt.xscale('log')
    plt.xlabel('C')
    plt.ylabel('Accuracy')
    plt.title('Train and Test Accuracy vs. C (Fixed Gamma)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Execution

In [11]:
# set solver settings
solvers.options["show_progress"] = False

# set mode to hide all non-required prints
hide_non_required_prints = True

In [12]:
# setup cross-validation space
kernel_functions = ["gaussian_kernel"]
kernel_parameters = np.logspace(-8, 4, 13)
C_values = np.logspace(-5, 3, 9)

# execute cross-validation
best_hyperparams, best_val_accuracy, best_train_accuracy = SVM_cross_validation(kernel_functions, kernel_parameters, C_values, k = 5)

print(f"\nBest hyperparameters: {best_hyperparams}")

100%|██████████| 117/117 [09:49<00:00,  5.04s/it]


Best hyperparameters: ('gaussian_kernel', 0.1, 1.0)


In [13]:
# plot of train and test accuracy vs C with fixed gamma

# non-required and hidden
if not hide_non_required_prints:
    ploy_varying_C(kernel_function = "gaussian_kernel",
                   gamma = 0.1,
                   C_values = np.logspace(-5, 3, 5000))

In [14]:
# plot of train and test accuracy vs gamma with fixed C

# non-required and hidden
if not hide_non_required_prints:
    plot_varying_gamma(kernel_function = "gaussian_kernel",
                       C = 1.0,
                       gammas = np.logspace(-8, 4, 5000))

In [15]:
# print values that are needed to compile the report but are not required
# to be printed on the notebook

# non-required and hidden
if not hide_non_required_prints:
    SVM_report(kernel_function = "gaussian_kernel",
               kernel_parameter = 0.1,
               C = 1.0)

In [16]:
# print values that are required to be printed on the notebook

SVM_print(kernel_function = "gaussian_kernel",
          kernel_parameter = 0.1,
          C = 1.0)

Setting values of the hyperparameters:
	C = 1.0
	Activation function = gaussian_kernel
	gamma = 0.1
Classification rate on the training set (% instances correctly classified): 0.9175
Classification rate on the test set (% instances correctly classified): 0.92
The confusion matrix:
[[94. 10.]
 [ 6. 90.]]
Time necessary for the optimization procedure: 1.5889891650000436 seconds
Number of optimization iterations: 13
Final difference between m(λ) and M(λ) (question 3): -0.009602091335748075
Final value of the objective function of the dual SVM problem: -141.57524979235575
